In [7]:
# ==========================================
# Phase 5 – LLM Responses Integration (Groq API)
# ==========================================

import os
import sys
from dotenv import load_dotenv
from groq import Groq

# ===============================
# 1. Load .env
# ===============================
load_dotenv()  # reads .env in project root

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
if not GROQ_API_KEY:
    raise ValueError("❌ Missing Groq API Key in .env. Please add GROQ_API_KEY=<your_key>.")

# ===============================
# 2. Imports from utils (main chatbot)
# ===============================
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from utils.chatbot import get_emotions, get_distress

# ===============================
# 3. Groq API client
# ===============================
client = Groq(api_key=GROQ_API_KEY)

# ===============================
# 4. Build Prompt
# ===============================
def build_prompt(user_input, emotions, distress):
    emo_str = ", ".join([f"{e} ({p:.2f})" for e, p in emotions]) if emotions else "None"
    dis_str = ", ".join([f"{d} ({p:.2f})" for d, p in distress]) if distress else "None"

    prompt = f"""
    You are a compassionate mental health chatbot.
    User message: "{user_input}"

    Detected emotions: {emo_str}
    Detected distress levels: {dis_str}

    Please respond empathetically in 2–3 sentences.
    If distress is critical, gently suggest reaching out to a professional.
    """
    return prompt.strip()

# ===============================
# 5. Generate LLM Response
# ===============================
def generate_llm_response(prompt: str) -> str:
    """
    Generate response using Groq API
    """
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",  # ✅ updated model
        messages=[{"role": "user", "content": prompt}],
        max_tokens=150,
        temperature=0.7
    )
    return response.choices[0].message.content

# ===============================
# 6. Test Chatbot with Groq LLM
# ===============================
def chatbot_response(user_input: str):
    emotions = get_emotions(user_input)
    distress = get_distress(user_input)

    prompt = build_prompt(user_input, emotions, distress)
    llm_reply = generate_llm_response(prompt)

    print("📝 User:", user_input)
    print("😊 Emotions:", emotions)
    print("⚠️ Distress:", distress)
    print("🤖 Bot:", llm_reply)
    return llm_reply

# ===============================
# 7. Example Run
# ===============================
if __name__ == "__main__":
    test_inputs = [
        "I feel so sad and lonely these days.",
        "I got a promotion at work and I’m really happy!",
        "Life feels meaningless, I don’t know if I can continue.",
        "I’m anxious about my exams tomorrow."
    ]

    for text in test_inputs:
        chatbot_response(text)
        print("=" * 60)


📝 User: I feel so sad and lonely these days.
😊 Emotions: [('sadness', 0.9953219294548035)]
⚠️ Distress: [('Depression', 0.3978669345378876)]
🤖 Bot: I can sense the weight of sadness and loneliness you're carrying, and it's totally okay to feel that way. It takes a lot of courage to acknowledge and express these emotions, and I'm here to listen and support you. Remember that you're not alone, and there are people who care about you and want to help.
📝 User: I got a promotion at work and I’m really happy!
😊 Emotions: [('joy', 0.9911966323852539), ('excitement', 0.2827065885066986)]
⚠️ Distress: [('Normal', 0.9051557779312134)]
🤖 Bot: That's fantastic news about your promotion! It's wonderful to hear that you're feeling joyful and excited about this new chapter in your career. Congratulations on this achievement - you must be feeling incredibly proud of yourself.
📝 User: Life feels meaningless, I don’t know if I can continue.
😊 Emotions: [('confusion', 0.7316586375236511), ('sadness', 0.6